In [43]:
# Install two dedicated libraries for handling jpeg2000 files, and a package for dicomsdl
!pip install --no-index --no-deps /kaggle/input/extrapackages-dicomsdl-gdcm-pylibjpeg/wheelhouse/dicomsdl-0.109.1-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl
!pip install --no-index --no-deps /kaggle/input/extrapackages-dicomsdl-gdcm-pylibjpeg/wheelhouse/numpy-1.21.6-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl
!pip install --no-index --no-deps /kaggle/input/extrapackages-dicomsdl-gdcm-pylibjpeg/wheelhouse/pylibjpeg-1.4.0-py3-none-any.whl
!pip install --no-index --no-deps /kaggle/input/extrapackages-dicomsdl-gdcm-pylibjpeg/wheelhouse/python_gdcm-3.0.21-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl


Processing /kaggle/input/extrapackages-dicomsdl-gdcm-pylibjpeg/wheelhouse/dicomsdl-0.109.1-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl
dicomsdl is already installed with the same version as the provided wheel. Use --force-reinstall to force an installation of the wheel.
Processing /kaggle/input/extrapackages-dicomsdl-gdcm-pylibjpeg/wheelhouse/numpy-1.21.6-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl
numpy is already installed with the same version as the provided wheel. Use --force-reinstall to force an installation of the wheel.
Processing /kaggle/input/extrapackages-dicomsdl-gdcm-pylibjpeg/wheelhouse/pylibjpeg-1.4.0-py3-none-any.whl
pylibjpeg is already installed with the same version as the provided wheel. Use --force-reinstall to force an installation of the wheel.
Processing /kaggle/input/extrapackages-dicomsdl-gdcm-pylibjpeg/wheelhouse/python_gdcm-3.0.21-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
python-gdcm is already installed with the 

In [44]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt # for making plots
from tqdm.notebook import tqdm
import sys
from joblib import Parallel, delayed
from multiprocessing import cpu_count
import dicomsdl

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import *
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator



# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

import glob

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [45]:
def fake_test_set(pd, pd_test):
    cols = pd.columns.difference(pd_test.columns).values.tolist()
    print(pd.columns)
    print(pd_test.columns)
    print(cols)
    pd['prediction_id'] = ''
    pd = pd[pd_test.columns]
    pd['prediction_id'] = pd.apply(
    lambda x: str(x['patient_id']) + '_' + x['laterality'], axis = 1
    )
    return pd

In [46]:
# Import csv data as a data frame.
csv_train = pd.read_csv("/kaggle/input/rsna-breast-cancer-detection/train.csv").iloc[:2000]
csv_test = pd.read_csv("/kaggle/input/rsna-breast-cancer-detection/test.csv")

train_images_folder = "/kaggle/input/rsna-breast-cancer-detection/train_images/"
debug = False
if debug:
    test_images_folder = "/kaggle/input/rsna-breast-cancer-detection/train_images/"
    csv_test_original = csv_test.copy()
    csv_test = fake_test_set(csv_train, csv_test)
    print(csv_test.columns)
    print(csv_train.columns)
    csv_test_original.head()
else:
    test_images_folder = "/kaggle/input/rsna-breast-cancer-detection/test_images/"

In [47]:
csv_test.head()

,site_id,patient_id,image_id,laterality,view,age,implant,machine_id,prediction_id
0,2,10008,736471439,L,MLO,81,0,21,10008_L
1,2,10008,1591370361,L,CC,81,0,21,10008_L
2,2,10008,68070693,R,MLO,81,0,21,10008_R
3,2,10008,361203119,R,CC,81,0,21,10008_R


In [48]:
# If any entries in the test set have missing entries in age, just set it to an avg.
data_test = csv_test.copy()

bad_ids = [1942326353]
csv_train = csv_train.drop(csv_train[csv_train['image_id'].isin(bad_ids)].index)
avg_age = csv_train['age'].mean()

In [49]:
def process_csv(df, cols_to_onehot = ['laterality', 'view'], avg_age = avg_age,
               pngfolder = '/kaggle/input/rsna-breast-cancer-detection/test_images/'):
    def onehot(df, encode):
        temp = pd.get_dummies(df[encode])
        df = df.drop([encode], axis=1)
        df = pd.concat([df, temp], axis=1)
        return df

    # Replace the small number of entries having missing age with the average age
    df['age'] = df['age'].fillna(avg_age)
    # Let's rescale so that ages listed are between (0, 1) for CNN convenience.
    df['age'] = df['age'] / 100

    # One-Hot: the Laterality and View columns are strings; there's only a few types of entries for each so let's just one-hot encode them.
    for col in cols_to_onehot:
        df = onehot(df, col)

    # Implant column is a int64 for some reason; let's convert it to uint8 so it's lighter.
    df['implant'] = df['implant'].astype('uint8')
    
    df['file'] = df.apply(
    lambda x: pngfolder + '/' + str(int(x['patient_id'])) + '/' + str(int(x['image_id'])) 
    + '.png', axis=1)
    
    return df

In [50]:
data_test = process_csv(data_test, avg_age = avg_age)
data_test.head()

,site_id,patient_id,image_id,age,implant,machine_id,prediction_id,L,R,CC,MLO,file
0,2,10008,736471439,0.81,0,21,10008_L,1,0,0,1,/kaggle/input/rsna-breast-cancer-detection/tes...
1,2,10008,1591370361,0.81,0,21,10008_L,1,0,1,0,/kaggle/input/rsna-breast-cancer-detection/tes...
2,2,10008,68070693,0.81,0,21,10008_R,0,1,0,1,/kaggle/input/rsna-breast-cancer-detection/tes...
3,2,10008,361203119,0.81,0,21,10008_R,0,1,1,0,/kaggle/input/rsna-breast-cancer-detection/tes...


In [51]:
# Put paths to all the images in a list to iterate through later

train_im_dir = train_images_folder
test_im_dir = test_images_folder

path = [test_im_dir + str(patient) + '/' + str(image) + '.dcm'
        for patient, image in zip(data_test['patient_id'], data_test['image_id'])]

In [52]:
import cv2
import time
import ray

def process_im_ray(im, image_size = 256, show = False):
    if show:
        try:
            print('begin process')
            plt.imshow(im.pixelData(), cmap='gray')
            plt.show(); plt.close()
        except: 
            print('begin process')
            plt.imshow(im.pixel_array, cmap='gray')
            plt.show(); plt.close()
    try:
        out = im.pixelData()
        phototype = im.getPixelDataInfo()['PhotometricInterpretation']
    except:
        out = im.pixel_array
        phototype = im.PhotometricInterpretation
        
    # MONOCHROME1 pictures use a backward brightness-darkness scheme from MONOCHROME2.
    # We reverse the MONOCHROME1 pictures to make this consistent.
    if phototype == 'MONOCHROME1':
        out = out.max() - out
        
    minval = out.min()
    maxval = out.max()
    
    # Crop image to interesting part.
    Threshold = maxval/10
    out = cut_empty_space_ray(out, T = Threshold, show = show)
    
    # scale up pixel vals so that pixel values lie between 0 and 1.
    out = (out-minval) / (maxval-minval)
    
    # Resize the photo.
    out = cv2.resize(out, (image_size, image_size))
    
    if show:
        plt.imshow(out, cmap='gray')
        plt.show(); plt.close()
        
    return out


def cut_empty_space_ray(im, T = 100, cutedge = 10, show = False):
    # ignore the border of images since some stray pixels there interfere with cropping.
    impx_cv2_raw = im[cutedge:-cutedge, cutedge:-cutedge] 
    
    # Convert image foreground % background to black and white.
    _, impx_cv2 = cv2.threshold(impx_cv2_raw, T, 255, cv2.THRESH_BINARY) 
    
    # Get edge detection contours
    contours, _ = cv2.findContours(impx_cv2.astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE) 
    boundary = max(contours, key=cv2.contourArea) # Get the contour enclosing the most space.
    
    # Make a mask so background is set to "color=0."
    # This eliminates image defects/non tissue such as labels, other random devices, etc
    impx_mask = np.zeros(impx_cv2.shape, dtype="uint8") 
    cv2.drawContours(impx_mask, [boundary], -1, 255, cv2.FILLED) 
    # Apply the mask on top of the original image.
    impx_cv2_masked = cv2.bitwise_and(impx_cv2_raw, impx_cv2_raw, mask = impx_mask) 
    
    # Get coordinates bounding the rectangle of the interesting parts of the image.
    s = time.time()
    x, y, w, h = cv2.boundingRect(boundary)

    
    ## Testing out how well the boundingRect function selects the correct region.
    #print(x, y, w, h)
    #blank = np.zeros(impx_cv2.shape, dtype="uint8")
    #m = np.amax(impx_cv2_masked)
    #cv2.rectangle((impx_cv2_masked),(x,y),(x+w,y+h), color = (int(m), 0, 0), thickness = 100, lineType=cv2.LINE_AA)
    #plt.imshow(impx_cv2_masked)
    #plt.show(); plt.close()
    
    out = impx_cv2_masked[y:y+h, x:x+w]
    if show:        
        plt.imshow(out)
        plt.show(); plt.close()
        print('end crop')
    return out

In [53]:
import multiprocessing
from joblib import Parallel, delayed

def get_images_new(imagepath, image_size = 256):
    
    im = dicomsdl.open(imagepath)
    image = process_im_ray(im, image_size)
    image = np.ndarray.flatten(image)
    del im
    return image
    
def save_images(imagepath, image_size = 256, savedir = None):
    
    im = dicomsdl.open(imagepath)
    image = process_im_ray(im, image_size)
    
    directory_split = imagepath.split('/')
    image_id = directory_split[-1].split('.')[0]
    patient_id = directory_split[-2]
    if savedir is not None:
        newfolder = savedir + '/' + patient_id
        fname = newfolder + '/' + image_id + '.png'
        os.makedirs(newfolder, exist_ok=True)
        image_save = (image * 255).astype(np.uint8)
        cv2.imwrite(fname, image_save)
    
def save_images_patient(patientpathlist, image_size = 256, savedir = None):
    patient_id = patientpathlist.split('/')[-1]
    patientimageslist = glob.glob(patientpathlist + '*.dcm')
    if savedir is not None:
        newfolder = savedir + '/' + patient_id
        os.makedirs(newfolder, exist_ok=True)
    for imagepath in patientimageslist:
        image = dicomsdl.open(imagepath)
        im_processed = process_im_ray(image)
        image_id = imagepath.split('/')[-1].split('.')[0]
        if savedir is not None:
            fname = newfolder + '/' + image_id + '.png'
            image_save = (im_processed * 255).astype(np.uint8)
            cv2.imwrite(fname, image_save)
        
    
start = time.time()
image_size = 512

run_dcm_to_png = True
show_prog_bar = False
endpoint = 100

if run_dcm_to_png:
    if debug:
        workdir = '/kaggle/working/train/'
    else:
        workdir = '/kaggle/working/test/'
    savedir = workdir + 'processed_' + str(image_size)
    num_cores = multiprocessing.cpu_count()
    
    processed_list = Parallel(n_jobs=num_cores)(delayed(save_images)(p, image_size = image_size,
                                                                    savedir = savedir) 
                                                        for p in tqdm(path))
#    pool = Multiprocessing.Pool(processes=4)
#    pool.map

#    ML_patientID_inputs_futures = [
#        save_images_patient.remote(p, image_size = image_size, savedir = save_dir) for p in patientpathlist[:100]
#    ]
#    ML_inputs = [x for x in tqdm(to_iterator(ML_patientID_inputs_futures), total = len(ML_patientID_inputs_futures))] ## Progress bar
    #ML_inputs = [x for x in (to_iterator(ML_patientID_inputs_futures))]

#Parallel(n_jobs = 4, require='sharedmem')(delayed(get_images_patient)(p, ML_inputs) for p in tqdm(patientpathlist[:125]))
print(time.time() - start)
#print(len(ML_inputs))
#print(len(ML_inputs[0]))

  0%|          | 0/4 [00:00<?, ?it/s]

0.8745970726013184


In [54]:
cols_1 = ['age', 'implant', 'L', 'R', 'AT', 'CC', 'LM', 'LMO', 'ML', 'MLO']
for c in cols_1:
    if c not in data_test.columns:
        data_test[c] = 0

data_test.head()

,site_id,patient_id,image_id,age,implant,machine_id,prediction_id,L,R,CC,MLO,file,AT,LM,LMO,ML
0,2,10008,736471439,0.81,0,21,10008_L,1,0,0,1,/kaggle/input/rsna-breast-cancer-detection/tes...,0,0,0,0
1,2,10008,1591370361,0.81,0,21,10008_L,1,0,1,0,/kaggle/input/rsna-breast-cancer-detection/tes...,0,0,0,0
2,2,10008,68070693,0.81,0,21,10008_R,0,1,0,1,/kaggle/input/rsna-breast-cancer-detection/tes...,0,0,0,0
3,2,10008,361203119,0.81,0,21,10008_R,0,1,1,0,/kaggle/input/rsna-breast-cancer-detection/tes...,0,0,0,0


In [55]:
class df_generator(tf.keras.utils.Sequence):
    def __init__(self, df, generator, directory = None, batch_size = 32, shuffle = True, 
                 subset = 'training', ImageSize = 512, columns = ['cancer'], 
                 test_set = 'False', **kwargs):
        self.df = df.copy()
        self.batch_size = batch_size
        self.count = 0
        self.shuffle = shuffle
        self.subset = subset
        self.columns = columns
        self.test_set = test_set
        self.df_flow = generator.flow_from_dataframe(
            self.df, directory=directory, shuffle = shuffle, 
            target_size = (ImageSize, ImageSize),
            subset = self.subset, batch_size = self.batch_size,
            x_col = 'file', y_col = self.columns, 
            class_mode = 'multi_output', color_mode = 'grayscale', 
            validate_filenames=False)
        self.n = self.df_flow.n
    
    def __len__(self):
        return int(np.ceil(self.n / self.batch_size))
    
    def on_epoch_end(self):
        self.df_flow.on_epoch_end()
        
    def reset(self):
        self.df_flow.reset()
        self.count = 0
    
    def __getitem__(self, idx = 0):
        data = self.df_flow.next()
        x_im = np.array(data[0]).astype(np.float16) # reference to image
        x_info = np.array(data[1][:-1]).T.astype(np.float16) # data columns
        y = data[1][-1]  # target column (i.e. cancer)
        self.count += self.batch_size
        if self.count > self.n:
            endidx = self.n % self.batch_size
            x_im = x_im[:endidx]
            x_info = x_info[:endidx]
            y = y[:endidx]
        if self.count >= self.n:
            self.reset()
        if self.test_set == True:
            return [[x_im, x_info]]
        else:
            return [x_im, x_info], y

In [56]:
if debug:
    pngfolder = '/kaggle/working/train/processed_512'
else:
    pngfolder = '/kaggle/working/test/processed_512'

# We'll add a column to the dataframe with all of the filenames to use with flow_from_dataframe.
data_test['file'] = data_test.apply(
    lambda x: pngfolder + '/' + str(int(x['patient_id'])) + '/' + str(int(x['image_id'])) 
    + '.png', axis=1)

#print(data_train_relevant.head)

# To create the correct shape of inputs for the network, we need to get both the
# image data and the csv data -- have all the columns be the "target y" for flow_from_dataframe.
def test_from_dataframe(directory, generator, subset='training', batch_size = 64,
                        data = data_test, columns = ['cancer'], seed = None):
    
    gendat = generator.flow_from_dataframe(data, directory=directory, shuffle = True, 
                                           target_size = (ImageSize, ImageSize),
                                           subset = subset, batch_size = batch_size,
                                           x_col = 'file', y_col = columns, 
                                           class_mode = 'multi_output', 
                                           color_mode = 'grayscale', 
                                           validate_filenames=False)
    N = gendat.n
    i = 0
    while i < N:
        data = gendat.next()
        x_im = np.array(data[0]).astype(np.float16) # reference to image
        x_info = np.array(data[1][:]).T.astype(np.float16) # data columns
        i += batch_size
        yield [[x_im, x_info]]
    return
ImageSize = 512 # i don't know if we should be resizing this
val_split = 0.0 
batch_size = 64 # No idea what a good number for this is, i'm using something close to what i saw on Google

cols = ['age', 'implant', 'L', 'R', 'AT', 'CC', 'LM', 'LMO', 'ML', 'MLO', 'cancer']
data_test['cancer'] = 0
# Make a Keras ImageDataGenerator
imagegen = ImageDataGenerator(rescale = 1./255., validation_split = 0)

test_gen = df_generator(data_test, imagegen, batch_size = batch_size, test_set = True,
                        columns = cols, shuffle = False)

Found 4 non-validated image filenames.


In [57]:
data_test.head()

,site_id,patient_id,image_id,age,implant,machine_id,prediction_id,L,R,CC,MLO,file,AT,LM,LMO,ML,cancer
0,2,10008,736471439,0.81,0,21,10008_L,1,0,0,1,/kaggle/working/test/processed_512/10008/73647...,0,0,0,0,0
1,2,10008,1591370361,0.81,0,21,10008_L,1,0,1,0,/kaggle/working/test/processed_512/10008/15913...,0,0,0,0,0
2,2,10008,68070693,0.81,0,21,10008_R,0,1,0,1,/kaggle/working/test/processed_512/10008/68070...,0,0,0,0,0
3,2,10008,361203119,0.81,0,21,10008_R,0,1,1,0,/kaggle/working/test/processed_512/10008/36120...,0,0,0,0,0


In [58]:
class F1Score(tf.keras.metrics.Metric):
    def __init__(self, name="f1", from_logits=False, **kwargs):
        super(F1Score, self).__init__(name=name, **kwargs)
        self.precision = tf.keras.metrics.Precision(from_logits)
        self.recall = tf.keras.metrics.Recall(from_logits)

    def update_state(self, y_true, y_pred, sample_weight=None):
        self.precision.update_state(y_true, y_pred, sample_weight)
        self.recall.update_state(y_true, y_pred, sample_weight)

    def result(self):
        p = self.precision.result()
        r = self.recall.result()
        return (2 * p * r) / (p + r + tf.keras.backend.epsilon())

    def reset_state(self):
        self.precision.reset_state()
        self.recall.reset_state()

In [59]:
from tensorflow.keras.metrics import *

input_CSV = tf.keras.layers.Input(shape=(10,))

input_Image = tf.keras.layers.Input(shape=(512,512, 1))

I0R = RandomRotation(0.4)(input_Image)
I0G = GaussianNoise(0.1)(I0R)
I0D = Dropout(0.1)(I0G)
image_Processed = I0D

I1C = Conv2D(32, 4, activation='relu', input_shape=(512, 512, 1))(image_Processed)
I1P = AveragePooling2D(pool_size=2)(I1C)
I3C = Conv2D(64, 4, activation='relu')(I1P)
I3P = AveragePooling2D(pool_size=2)(I3C)
I3D = Dropout(0.1)(I3P)
I4C = Conv2D(128, 4, activation='relu')(I3D)
I4P = AveragePooling2D(pool_size=2)(I4C)
I4D = Dropout(0.1)(I4P)
I5P = AveragePooling2D(pool_size=2)(I4D)
I6 = Flatten()(I5P)

input_all = Concatenate()([input_CSV, I6])
CNN = Dense(256, activation='relu')(input_all)
CNN = Dense(64, activation='relu')(CNN)
CNN = Dense(1, activation='sigmoid')(CNN)

ML = Model(inputs=[input_Image, input_CSV], outputs=CNN)
ML.summary()

# seems to be the right loss function - used for binary outcomes
# I set from_logits to True because I'm not assuming that things are standardized. It generally seems like a safer option

# decreased learning rate significantly to avoid immediately finding local minimum
optimizer = tf.keras.optimizers.Adam(
    learning_rate=1e-5, weight_decay = 1e-6
)

from_logits = False
ML.compile(optimizer=optimizer,
           loss='binary_crossentropy',
           metrics=[tf.keras.metrics.TruePositives(),
                    tf.keras.metrics.FalsePositives(),
                    tf.keras.metrics.TrueNegatives(),
                    tf.keras.metrics.FalseNegatives(),
                    F1Score()]
          )

model_import_path = '/kaggle/input/npd-bcp-B/npd_bcp_B.h5'
ML.load_weights(model_import_path)

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_6 (InputLayer)           [(None, 512, 512, 1  0           []                               
                                )]                                                                
                                                                                                  
 random_rotation_2 (RandomRotat  (None, 512, 512, 1)  0          ['input_6[0][0]']                
 ion)                                                                                             
                                                                                                  
 gaussian_noise_2 (GaussianNois  (None, 512, 512, 1)  0          ['random_rotation_2[0][0]']      
 e)                                                                                         

In [60]:
test_pred = ML.predict(test_gen)

1/1 [==============================] - 1s 590ms/step


In [61]:
test_pred_series = pd.Series(test_pred.T[0])
predictions = pd.DataFrame()
predictions['image_id'] = pd.Series(data_test['image_id']).tolist()
predictions['cancer'] = (test_pred_series).tolist()
predict_test = csv_test.copy()
predict_test['cancer'] = (test_pred_series).tolist()
print(predict_test)

   site_id  patient_id    image_id laterality view  age  implant  machine_id  \
0        2       10008   736471439          L  MLO   81        0          21   
1        2       10008  1591370361          L   CC   81        0          21   
2        2       10008    68070693          R  MLO   81        0          21   
3        2       10008   361203119          R   CC   81        0          21   

  prediction_id    cancer  
0       10008_L  0.523676  
1       10008_L  0.531976  
2       10008_R  0.497824  
3       10008_R  0.533144  


In [62]:
predict_merge = predict_test.groupby(['patient_id', 'laterality'])['cancer'].max()
print(predict_merge)
predict_merge = predict_merge
predict_merge = predict_merge.reset_index()
predict_merge['prediction_id'] = predict_merge.apply(
    lambda x: str(x['patient_id']) + '_' + x['laterality'], axis = 1
)
print(predict_merge)
output_cols = ['predict_merge', 'cancer']
predict_merge = predict_merge.reindex(columns=['prediction_id', 'cancer'])
print(predict_merge)

patient_id  laterality
10008       L             0.531976
            R             0.533144
Name: cancer, dtype: float64
   patient_id laterality    cancer prediction_id
0       10008          L  0.531976       10008_L
1       10008          R  0.533144       10008_R
  prediction_id    cancer
0       10008_L  0.531976
1       10008_R  0.533144


In [63]:
predict_merge.to_csv('submission.csv', index=False)